In [1]:
%load_ext autoreload
%autoreload 2

## Open questions

* How to show all the resulting contexts for all examples to the user?
* How to show the intermediates to the user?
* How to allow the user to compare two prompt versions?
* How to allow the user to provide feedback on whether something is a good result or not?
* How to manage the version blowup when automatically saving each .run() result as a version?

In [2]:
%load_ext gears.extension 
%reload_ext gears.extension

In [3]:
from gears import Example, Session, Gear, History
from gears.llms import AzureOpenAIChat
from gears.utils import extract_first_json
from dotenv import load_dotenv
import openai
import os

# Set Azure keys
load_dotenv()
openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_AZURE_API_KEY")
openai.api_version = "2023-07-01-preview"

In [4]:
class Context(Example):
    city: str

In [5]:
session = Session()

session.add_example(Context(city="Paris"))

In [6]:
session.examples

[Context(id='d3ee24cc-d899-433e-96f0-4663d8fee658', city='Paris')]

In [7]:
%%gear --root
class Ideation(Gear):
    def __init__(self):
        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))
    
    def template(self, context: Context):
        return "Suggest some types of interactions that one might encounter when visiting the city {{ context.city }} as a tourist. Return a JSON with key `interactions` and value equal to a list of strings."
    
    def transform(self, reply: dict, context: Context):
        interactions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["interactions"]
        context.interactions = interactions
        
        return context
    
    def switch(self, context: Context):
        return EnglishQuestion()

In [8]:
%%gear
class EnglishQuestion(Gear):
    def __init__(self):
        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))
    
    def template(self, context: Context):
        return "For each interaction, write a question that a tourist might ask about the interaction. Return a JSON with key `questions` and value equal to a list of strings."
    
    def transform(self, reply: dict, context: Context):
        questions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["questions"]
        context.questions = questions
        
        return context

    def switch(self, context: Context):
        return LanguageQuestion()

In [9]:
%%gear
class LanguageQuestion(Gear):
    def __init__(self):
        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))
    
    def template(self, context: Context):
        return "Consider the most popular language in the city {{ context.city }}. Then translate each question into this language. If the most popular language of the city is English, then return the same English questions. Return a JSON with key `translations` and value equal to a list of strings."
    
    def transform(self, reply: dict, context: Context):
        translated_questions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["translations"]
        context.translated_questions = translated_questions
        
        return context

In [11]:
# Run session

await session.run(version=True)

[00:27:12] INFO     Running LanguageQuestion on example d3ee24cc-d899-433e-96f0-4663d8fee658.        ]8;id=451672;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=783339;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#143\143]8;;\

[00:27:48] INFO     message='OpenAI API response'                                                        ]8;id=959362;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py\util.py]8;;\:]8;id=856603;file:///Users/shreyashankar/miniforge3/envs/gearstest/lib/python3.9/site-packages/openai/util.py#67\67]8;;\
                    path=https://shreyashankar.openai.azure.com//openai/deployments/gpt-35-turbo/chat/co           
                    mpletions?api-version=2023-07-01-preview processing_ms=34883.7059                              
                    request_id=1826efdc-b91c-442e-8eeb-3ad43793e730 response_code=200                              

           INFO     This run cost $0.0026. The total cost of the session so far is $0.0034.          ]8;id=73576;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=810940;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#203\203]8;;\

           INFO     Saving workflow 1 to session.                                                    ]8;id=618172;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=913444;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#228\228]8;;\

[Context(id='d3ee24cc-d899-433e-96f0-4663d8fee658', city='Paris', interactions=['Ordering food at a restaurant', 'Asking for directions', 'Shopping at a market or boutique', 'Visiting museums and art galleries', 'Taking a guided tour', 'Attending a cultural event or performance', 'Haggling at a flea market', 'Meeting locals and practicing language skills', 'Riding the metro or using public transportation', 'Exploring historic landmarks and architecture'], questions=['What are some traditional French dishes that I should try?', 'How do I get to the Eiffel Tower?', 'What are some popular souvenirs to bring back from Paris?', 'What are the hours of operation for the Louvre?', 'Can you recommend a good tour company?', 'What festivals are happening during my visit?', 'How do I negotiate prices at a flea market?', 'Where can I meet locals and practice my French?', 'Is it safe to use public transportation at night?', 'What are some must-see landmarks in Paris?'], translated_questions=['Quels 

In [ ]:
# Run session again (should not ping OpenAI API)

await session.run()

           INFO     No new gears ran. The total cost of the session so far is $0.0020.               ]8;id=473031;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=202854;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#207\207]8;;\

[Context(id='64ca6332-65d8-4c88-911e-1895baa66b36', city='Paris', interactions=['Ordering coffee in a café', 'Asking for directions to a famous landmark', 'Buying souvenirs from a street vendor', 'Chatting with locals in a park', 'Taking a guided tour of the Louvre', 'Attending a French cooking class', 'Shopping at a flea market', 'Having a picnic in front of the Eiffel Tower', 'Joining a group for a wine tasting', 'Watching a street performer'], questions=['How do I order coffee in a café?', 'Excuse me, could you tell me how to get to the Eiffel Tower?', 'What kind of souvenirs do you have?', "Do you come to this park often? What's your favorite thing to do here?", 'What are some must-see artworks at the Louvre?', 'What are some classic French dishes that I should learn to cook?', 'What kind of items can I find at the flea market?', 'Is it okay to have a picnic here? Are there any rules or regulations we should keep in mind?', 'What are some good local wines to try?', 'How long have y

In [15]:
# Add an example

session.add_example(Context(city="London"))

In [ ]:
# Run session again (should ping OpenAI API only once, for the new example)

await session.run(version=True)

[00:24:13] INFO     No new gears ran. The total cost of the session so far is $0.0044.               ]8;id=868349;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=80003;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#207\207]8;;\

{'Ideation': 'class Ideation(Gear):\n    def __init__(self):\n        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))\n    \n    def template(self, context: Context):\n        return "Suggest some types of interactions that one might encounter when visiting the city {{ context.city }} as a tourist. Return a JSON with key `interactions` and value equal to a list of strings."\n    \n    def transform(self, reply: dict, context: Context):\n        interactions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["interactions"]\n        context.interactions = interactions\n        \n        return context\n    \n    def switch(self, context: Context):\n        return EnglishQuestion()\n', 'EnglishQuestion': 'class EnglishQuestion(Gear):\n    def __init__(self):\n        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))\n    \n    def template(self, context: Context):\n        return "For each interaction, write a question that a tourist mig

           INFO     Saving workflow 2 to session.                                                    ]8;id=840347;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=465238;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#230\230]8;;\

[Context(id='64ca6332-65d8-4c88-911e-1895baa66b36', city='Paris', interactions=['Ordering coffee in a café', 'Asking for directions to a famous landmark', 'Buying souvenirs from a street vendor', 'Chatting with locals in a park', 'Taking a guided tour of the Louvre', 'Attending a French cooking class', 'Shopping at a flea market', 'Having a picnic in front of the Eiffel Tower', 'Joining a group for a wine tasting', 'Watching a street performer'], questions=['How do I order coffee in a café?', 'Excuse me, could you tell me how to get to the Eiffel Tower?', 'What kind of souvenirs do you have?', "Do you come to this park often? What's your favorite thing to do here?", 'What are some must-see artworks at the Louvre?', 'What are some classic French dishes that I should learn to cook?', 'What kind of items can I find at the flea market?', 'Is it okay to have a picnic here? Are there any rules or regulations we should keep in mind?', 'What are some good local wines to try?', 'How long have y

In [ ]:
session.cost

0.004371

In [12]:
session.versions[0]

{'root': 'Ideation',
 'code': {'Ideation': 'class Ideation(Gear):\n    def __init__(self):\n        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))\n    \n    def template(self, context: Context):\n        return "Suggest some types of interactions that one might encounter when visiting the city {{ context.city }} as a tourist. Return a JSON with key `interactions` and value equal to a list of strings."\n    \n    def transform(self, reply: dict, context: Context):\n        interactions = extract_first_json(reply["choices"][0]["message"]["content"].strip())["interactions"]\n        context.interactions = interactions\n        \n        return context\n    \n    def switch(self, context: Context):\n        return EnglishQuestion()\n',
  'EnglishQuestion': 'class EnglishQuestion(Gear):\n    def __init__(self):\n        super().__init__(AzureOpenAIChat(deployment_id="gpt-35-turbo"))\n    \n    def template(self, context: Context):\n        return "For each interaction, wri

In [13]:
session.add_example(Context(city="Berlin"))

In [18]:
import pandas as pd

res = await session.run()
display(pd.DataFrame(res))

[14:25:31] INFO     No new gears ran. The total cost of the session so far is $0.0077.               ]8;id=783154;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py\session.py]8;;\:]8;id=612702;file:///Users/shreyashankar/Documents/projects/gears/gears/session.py#195\195]8;;\

,0,1,2,3,4
0,"(id, d3ee24cc-d899-433e-96f0-4663d8fee658)","(city, Paris)","(interactions, [Ordering food at a restaurant,...","(questions, [What are some traditional French ...","(translated_questions, [Quels plats traditionn..."
1,"(id, d3ee24cc-d899-433e-96f0-4663d8fee658)","(city, Berlin)","(interactions, [Tour of the Berlin Wall, Visit...","(questions, [What is the history of the Berlin...","(translated_questions, [Was ist die Geschichte..."
2,"(id, d3ee24cc-d899-433e-96f0-4663d8fee658)","(city, London)","(interactions, [Touring famous landmarks such ...","(questions, [What is the history behind the la...","(translated_questions, [What is the history be..."


In [20]:
pd.DataFrame([r.dict() for r in res])

/var/folders/nq/ldkhrrws0xb9whw7b6rpzhc00000gn/T/ipykernel_23755/113781542.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.0.3/migration/
  pd.DataFrame([r.dict() for r in res])


,id,city,interactions,questions,translated_questions
0,d3ee24cc-d899-433e-96f0-4663d8fee658,Paris,"[Ordering food at a restaurant, Asking for dir...",[What are some traditional French dishes that ...,[Quels plats traditionnels français devrais-je...
1,d3ee24cc-d899-433e-96f0-4663d8fee658,Berlin,"[Tour of the Berlin Wall, Visit to Brandenburg...","[What is the history of the Berlin Wall?, What...","[Was ist die Geschichte der Berliner Mauer?, W..."
2,d3ee24cc-d899-433e-96f0-4663d8fee658,London,"[Touring famous landmarks such as Big Ben, the...","[What is the history behind the landmark?, Wha...","[What is the history behind the landmark?, Wha..."
